<a href="https://colab.research.google.com/github/LalithShankar/sentiment-analysis/blob/master/Plain_Text_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import re
import textract
#read the content of pdf as text
text1 = textract.process('UBS Financial Services proceedings.pdf').decode("utf-8")
#use four space as paragraph delimiter to convert the text into list of paragraphs.
type(text1)
text1
x = re.split('\s{3,}', text1)
x

In [0]:
pip install textract

In [0]:
x1 = re.split('.\n', text1)
x1

In [25]:
def listtostring(s):
  str1 = "\n"
  return (str1.join(s))
l2s= listtostring(x)
l2s
print(l2s)

ADMINISTRATIVE PROCEEDING 
File No. 3-18270
SEC Charges UBS in Connection with Mutual Fund Sales to Retirement and Charitable 
Accounts
October 27, 2017 – The Securities and Exchange Commission today announced that UBS 
Financial Services Inc. (UBS), a registered broker-dealer and investment adviser located in New 
York, New York, agreed to settle charges that it failed to provide certain retail retirement and 
charitable brokerage accounts with sales charge waivers and/or lower fee share classes when 
selling certain mutual funds.
According to the SEC’s order, from at least January 2010 through June 2015, UBS 
disadvantaged certain retirement account and charitable organization customers by failing to 
ascertain their eligibility for a less expensive mutual fund share class, and recommending and 
selling them more expensive mutual fund share classes, when less expensive share classes were 
available.  UBS did so without disclosing that it would receive greater compensation from the 
c

The whole document as DataFrame

In [0]:
n = l2s.replace('\n', '')
finallist = n.split('                     ')

In [92]:
data = {'Name':finallist}
data

{'Name': ['ADMINISTRATIVE PROCEEDING File No. 3-18270SEC Charges UBS in Connection with Mutual Fund Sales to Retirement and Charitable AccountsOctober 27, 2017 – The Securities and Exchange Commission today announced that UBS Financial Services Inc. (UBS), a registered broker-dealer and investment adviser located in New York, New York, agreed to settle charges that it failed to provide certain retail retirement and charitable brokerage accounts with sales charge waivers and/or lower fee share classes when selling certain mutual funds.According to the SEC’s order, from at least January 2010 through June 2015, UBS disadvantaged certain retirement account and charitable organization customers by failing to ascertain their eligibility for a less expensive mutual fund share class, and recommending and selling them more expensive mutual fund share classes, when less expensive share classes were available.  UBS did so without disclosing that it would receive greater compensation from the cust

In [93]:
df = pd.DataFrame(data)
 
print(df)
type(df)

                                                Name
0  ADMINISTRATIVE PROCEEDING File No. 3-18270SEC ...


pandas.core.frame.DataFrame

In [0]:
from textblob import TextBlob

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity


In [95]:
polarity = df['Name'].apply(pol)
# type(polarity)
subject = df['Name'].apply(sub)

print(polarity)
print(subject)

0    0.025896
Name: Name, dtype: float64
0    0.525186
Name: Name, dtype: float64


In [101]:
frame = {'Data':[finallist]}
dataframe = pd.DataFrame(zip(finallist,polarity,subject),columns=['Data','Polarity','Subjectivity'])
dataframe

,Data,Polarity,Subjectivity
0,ADMINISTRATIVE PROCEEDING File No. 3-18270SEC ...,0.025896,0.525186


In [0]:
dataframe.to_csv('Rule_Based_Sentiment_UBS.csv', index_label='index')



In [0]:
pip install shorttext

In [0]:
import shorttext
import numpy as np
import pandas as pd
import re

In [69]:
from stemming.porter import stem

SyntaxError: ignored

In [104]:
pipeline = [lambda s: re.sub('[^\w\s]', '', s),
            lambda s: re.sub('[\d]', '', s),
            lambda s: s.lower(),
            
 ]
txtpreproceesor = shorttext.utils.text_preprocessor(pipeline)
txtpreproceesor

<function shorttext.utils.textpreprocessing.text_preprocessor.<locals>.<lambda>>

In [105]:
docx = pd.read_csv('Rule_Based_Sentiment_UBS.csv')
docx

,index,Data,Polarity,Subjectivity
0,0,ADMINISTRATIVE PROCEEDING File No. 3-18270SEC ...,0.025896,0.525186


Topic Modelling

In [0]:
docids = list(docx['index'])
corpus = [txtpreproceesor(Data).split(' ') for Data in docx['Data']]
# corpus = [txtpreproceesor(Data).split(' ') for Data in usprezdf['Data']]
corpus

In [107]:
dtm = shorttext.utils.DocumentTermMatrix(corpus, docids=docids, tfidf=False)
print(dtm)

In [0]:
pip install gensim

In [0]:
from gensim import matutils, models
import scipy.sparse

In [0]:
tdm = docx.transpose()
tdm.head()

In [0]:
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [112]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [113]:
data_nouns = pd.DataFrame(docx.Data.apply(nouns))
data_nouns

,Data
0,PROCEEDING File No Charges UBS Connection Mutu...


In [114]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.Data)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,according,account,accounts,accountsoctober,act,addresses,adviser,analysis,analytics,asset,assistant,broker,brokerage,censure,charge,charges,charitable,class,classes,commission,compensation,compliance,connection,contingent,cost,customer,customers,data,dealer,director,division,efforts,eligibility,enforcement,examination,examinations,exchange,expenses,failures,farinacci,...,investigation,investment,investments,january,john,june,licardo,light,management,mutual,new,office,ongoing,order,organization,payments,penalty,proceeding,purchases,result,retirement,return,risk,sales,sec,sections,securities,services,share,structures,support,szczepanik,team,today,total,ubs,unit,valerie,waivers,york
0,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,4,1,2,4,1,1,1,1,1,1,1,8,1,1,1,1,1,1,1,1,1,1,2,1,1,...,1,1,1,1,1,1,1,1,1,1,2,1,1,4,1,2,1,2,2,1,3,1,1,4,4,1,2,1,8,1,1,1,1,1,1,10,1,1,1,2


In [0]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())
id2wordn

topics with noun

In [116]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.012*"ubs" + 0.012*"share" + 0.012*"customers" + 0.011*"charges" + 0.011*"order" + 0.011*"classes" + 0.011*"fund" + 0.011*"sales" + 0.011*"york" + 0.011*"sec"'),
 (1,
  '0.056*"ubs" + 0.045*"customers" + 0.045*"share" + 0.029*"fund" + 0.024*"sec" + 0.024*"sales" + 0.024*"classes" + 0.024*"order" + 0.024*"charges" + 0.019*"retirement"')]

In [117]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.012*"ubs" + 0.012*"share" + 0.011*"fund" + 0.011*"customers" + 0.011*"sec" + 0.011*"sales" + 0.011*"charges" + 0.011*"retirement" + 0.011*"order" + 0.011*"classes"'),
 (1,
  '0.012*"ubs" + 0.012*"customers" + 0.012*"share" + 0.012*"fund" + 0.012*"sales" + 0.011*"order" + 0.011*"classes" + 0.011*"sec" + 0.011*"charges" + 0.011*"retirement"'),
 (2,
  '0.062*"ubs" + 0.050*"share" + 0.050*"customers" + 0.032*"fund" + 0.026*"order" + 0.026*"charges" + 0.026*"sec" + 0.026*"classes" + 0.026*"sales" + 0.020*"retirement"'),
 (3,
  '0.012*"customers" + 0.012*"ubs" + 0.012*"share" + 0.012*"classes" + 0.011*"fund" + 0.011*"sales" + 0.011*"sec" + 0.011*"retirement" + 0.011*"charges" + 0.011*"order"')]

noun abd adjective

In [0]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [121]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(docx.Data.apply(nouns_adj))
data_nouns_adj

,Data
0,ADMINISTRATIVE PROCEEDING File No 3-18270SEC C...


In [127]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words)
data_cvna = cvna.fit_transform(data_nouns_adj.Data)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,18270sec,according,account,accounts,accountsoctober,act,additional,addresses,administrative,adviser,analysis,analytics,asset,assistant,available,broker,brokerage,cease,censure,certain,charge,charges,charitable,civil,class,classes,commission,compensation,compliance,connection,contingent,cost,customer,customers,data,dealer,desist,different,director,division,...,licardo,light,lower,lowest,management,mutual,new,office,ongoing,order,organization,overall,payments,penalty,proceeding,purchases,registered,result,retail,retirement,return,risk,sales,sec,sections,securities,services,settled,share,structures,support,szczepanik,team,today,total,ubs,unit,valerie,waivers,york
0,1,1,1,2,1,1,1,1,2,1,1,1,1,1,1,1,1,2,1,3,1,4,3,1,2,4,1,1,1,1,1,1,1,8,1,1,2,2,1,1,...,1,1,1,1,1,5,2,1,1,4,1,1,2,1,2,2,1,1,1,3,1,1,4,4,1,2,1,1,8,1,1,1,1,1,1,10,1,1,1,2


In [0]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [129]:
# Let's start with 2 topics


[(0,
  '0.045*"ubs" + 0.036*"share" + 0.036*"customers" + 0.024*"expensive" + 0.024*"fund" + 0.024*"mutual" + 0.019*"classes" + 0.019*"sec" + 0.019*"sales" + 0.019*"charges"'),
 (1,
  '0.010*"ubs" + 0.010*"customers" + 0.010*"share" + 0.010*"mutual" + 0.009*"order" + 0.009*"fund" + 0.009*"expensive" + 0.009*"charges" + 0.009*"sales" + 0.009*"sec"')]

In [153]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=10, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.009*"ubs" + 0.009*"fund" + 0.009*"share" + 0.009*"customers" + 0.009*"expensive" + 0.009*"mutual" + 0.009*"charges" + 0.009*"sec" + 0.009*"sales" + 0.009*"certain"'),
 (1,
  '0.010*"customers" + 0.010*"share" + 0.010*"ubs" + 0.009*"mutual" + 0.009*"sec" + 0.009*"fund" + 0.009*"charges" + 0.009*"expensive" + 0.009*"sales" + 0.009*"classes"'),
 (2,
  '0.014*"share" + 0.014*"customers" + 0.013*"ubs" + 0.011*"mutual" + 0.011*"expensive" + 0.011*"order" + 0.011*"fund" + 0.011*"sales" + 0.010*"retirement" + 0.010*"sec"'),
 (3,
  '0.009*"ubs" + 0.009*"customers" + 0.009*"share" + 0.009*"sec" + 0.009*"fund" + 0.009*"expensive" + 0.009*"mutual" + 0.009*"order" + 0.009*"sales" + 0.009*"charges"'),
 (4,
  '0.010*"ubs" + 0.010*"share" + 0.009*"customers" + 0.009*"sec" + 0.009*"fund" + 0.009*"expensive" + 0.009*"certain" + 0.009*"mutual" + 0.009*"order" + 0.009*"charges"'),
 (5,
  '0.010*"share" + 0.010*"customers" + 0.010*"ubs" + 0.009*"fund" + 0.009*"expensive" + 0.009*"mutual" + 0.009*

In [162]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=10, id2word=id2wordna, passes=10)


ldana.top_topics(corpus=corpusna, topn=10, processes=1)

[([(0.009456851, 'ubs'),
   (0.009410341, 'customers'),
   (0.00936581, 'share'),
   (0.009354374, 'sales'),
   (0.009341937, 'charges'),
   (0.009335789, 'mutual'),
   (0.009333886, 'expensive'),
   (0.009327227, 'classes'),
   (0.009320822, 'sec'),
   (0.009296105, 'order')],
  1.0000889005818412e-12),
 ([(0.0093558505, 'ubs'),
   (0.009348861, 'share'),
   (0.0093427645, 'customers'),
   (0.009309359, 'mutual'),
   (0.009301057, 'sales'),
   (0.009297199, 'fund'),
   (0.009296582, 'expensive'),
   (0.009290089, 'order'),
   (0.009287105, 'charges'),
   (0.009285108, 'sec')],
  1.0000889005818412e-12),
 ([(0.0099852225, 'ubs'),
   (0.009686532, 'customers'),
   (0.0095775565, 'share'),
   (0.0095188245, 'sales'),
   (0.009510258, 'mutual'),
   (0.009496671, 'order'),
   (0.009462416, 'expensive'),
   (0.009455371, 'fund'),
   (0.009431598, 'sec'),
   (0.009415428, 'charges')],
  1.0000889005818412e-12),
 ([(0.053199947, 'ubs'),
   (0.042663023, 'customers'),
   (0.042659275, 'share')

In [0]:

# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
d = list(zip([a for [(a, b)] in corpus_transformed], data_dtmna.index))

In [132]:
d

[(1, 0)]